# 一、简介

在股票行情分析中，我们经常使用 EMA、SMMA 等指标，这也是社区中经常被提及的一点，  
于是这里我们单开一个帖子来讲解 EMA 和 SMMA 的实现。  

EMA 是指数平均数指标，英文 Exponential Moving Average；  
而这里的 SMMA 指平滑移动平均线（而不是简单移动平均线 SMA 或 MA），英文 Smoothed Moving Average。

# 二、算法

这边其实不用很关注，不感兴趣的略过即可。

## 先看 EMA 是如何计算的：

    EMA = (CLOSE(i) * P) + (EMA(i - 1) * (100 - P))

    Where:
    CLOSE(i) – the price of the current period closure;
    EMA(i-1) – Exponentially Moving Average of the previous period closure;
    P – the percentage of using the price value.

## 再来看一下 SMMA 是如何计算的：

    The first value of this smoothed moving average is calculated as the simple moving average (SMA):
    SUM1 = SUM(CLOSE, N)
    SMMA1 = SUM1/N
    
    The second and succeeding moving averages are calculated according to this formula:
    PREVSUM = SMMA(i - 1) * N
    SMMA(i) = (PREVSUM - SMMA(i - 1) + CLOSE(i)) / N
    
    Where:
    SUM1 – is the total sum of closing prices for N periods;
    PREVSUM – smoothed sum of previous bar;
    SMMA1 – is the smoothed moving average of the first bar;
    SMMA(i) – is the smoothed moving average of the current bar (except for the first one);
    CLOSE(i) – is the current closing price;
    N – is the smoothing period.
    
    The formula can be simplified as a result of arithmetic manipulations:
    SMMA (i) = (SMMA(i - 1) * (N - 1) + CLOSE (i)) / N

# 三、用 talib 实现

Python 中做各种平均计算时，我们经常引用 talib。  
talib 支持的平均种类如下：

    MA_Type: 0=SMA, 1=EMA, 2=WMA, 3=DEMA, 4=TEMA, 5=TRIMA, 6=KAMA, 7=MAMA, 8=T3 (Default=SMA)
    
许多人误以为 talib 中的 SMA 跟上面提及的 SMMA 是等同的，其实不然。  
其实 talib 中的 SMA 是 Simple Moving Average，等同于平时在股票行情软件的经常看到的 MA，也就是用 Python 的 mean() 可以直接计算得到的。

## EMA 的实现

### 试试手
先构造一组 0-20 的数组：

In [1]:
import numpy as np
x = np.arange(20.)
x

array([  0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,
        11.,  12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.])

然后计算 EMA

In [2]:
import talib
EMA = talib.EMA(x, 5)
EMA

array([ nan,  nan,  nan,  nan,   2.,   3.,   4.,   5.,   6.,   7.,   8.,
         9.,  10.,  11.,  12.,  13.,  14.,  15.,  16.,  17.])

### 然后我们自己把 EMA 写出来

为了和 talib 保持一致，我们改写 

    EMA = (CLOSE(i) * P) + (EMA(i - 1) * (100 - P))
    
令：   

    P = 2 ／ (n + 1)
    100 - P = (n - 1) / (n + 1)

In [3]:
def test_EMA(close, n):
    result = np.array([np.nan] * len(close))
    result[n - 1] = close[:n].mean()
    for i in range(n, len(close)):
        result[i] = (2 * close[i] + (n - 1) * result[i - 1]) / (n + 1)
    return result

拿一组数据来看看和 talib 的 EMA 一样不：

In [4]:
close = get_price('000001.XSHE', start_date='20170601', end_date='20170701', frequency='1d')['close'].values

In [5]:
close

array([ 9.0568,  9.0371,  8.8991,  8.909 ,  8.9977,  8.9977,  9.0174,
        8.978 ,  8.9878,  8.9484,  8.909 ,  8.8893,  8.9977,  8.9878,
        9.0174,  9.1159,  9.1159,  9.1652,  9.2243,  9.2933,  9.2933,
        9.2539])

In [6]:
talib.EMA(close, 18)

array([        nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,         nan,  9.00151111,  9.02496257,  9.05320862,
        9.0784814 ,  9.09694651])

In [7]:
np.array(test_EMA(close, 18))

array([        nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,         nan,  9.00151111,  9.02496257,  9.05320862,
        9.0784814 ,  9.09694651])

完全一致，这样 EMA 就实现了。

## SMMA 的实现

### 根据等式计算 

根据实际中 SMMA 的运用，我们有等式： 

    SMMA(close, n, 2) = EMA(close, n - 1) 
    
这样我们可以很容易地计算出 SMMA 来。

In [8]:
def SMMA(close, n, m=2):
    result = talib.EMA(close, n - 1)
    return result

In [9]:
test_EMA(close, 18)

array([        nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,         nan,  9.00151111,  9.02496257,  9.05320862,
        9.0784814 ,  9.09694651])

In [10]:
SMMA(close, 19, m=2)

array([        nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,         nan,  9.00151111,  9.02496257,  9.05320862,
        9.0784814 ,  9.09694651])

### 根据第二部分的算法，我们也可以徒手写出 SMMA 的函数

In [11]:
def test_SMMA(close, n, m=2):
    result = np.array([np.nan] * len(close))
    result[n - 2] = close[:n - 1].mean()
    for i in range(n - 1, len(close)):
        result[i] = (result[i - 1] * (n - 2) + 2 * close[i]) / n
    return result

In [12]:
test_SMMA(close, 19, m=2)

array([        nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,         nan,  9.00151111,  9.02496257,  9.05320862,
        9.0784814 ,  9.09694651])

# 最后

其实很简单，  
其实很自然，  
多写就会了，，

PS： 
本文算法部分摘自 [Moving Average](https://www.metatrader4.com/en/trading-platform/help/analytics/tech_indicators/moving_average)，关于 Moving Average 的常用算法在这里都可以找到。  
